In [1]:
import os, sys, glob
import pandas as pd
import numpy as np
import pickle
import warnings
import math
# from process_mmrf import get_sequential_tensor, merge_on_pids, parse_baseline, parse_outcomes, parse_treatments, parse_labs, parse_trt_outcomes
from fancyimpute import KNN as KNN_impute

Using TensorFlow backend.
/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [2]:
ia_version = 'ia15'
if ia_version == 'ia13':
    FDIR  = '/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia13/CoMMpass_IA13_FlatFiles'
elif ia_version == 'ia15': 
    FDIR  = '/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia15/CoMMpass_IA15_FlatFiles'
else:
    raise ValueError('Bad ia version')

In [3]:
data_files = {}
for fullname in glob.glob(FDIR+'/*.csv'):
    print (fullname)
    fname = os.path.basename(fullname).split('.')[0]
    if 'MMRF_CoMMpass_IA13_' in fname:
        kname = fname.split('MMRF_CoMMpass_IA13_')[1]
    elif 'MMRF_CoMMpass_IA15_' in fname: 
        kname = fname.split('MMRF_CoMMpass_IA15_')[1]
    else:
        kname = fname
    data_files[kname] = pd.read_csv(fullname, delimiter=',', encoding='latin-1')
print (data_files.keys())

/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia15/CoMMpass_IA15_FlatFiles/MMRF_CoMMpass_IA15_STAND_ALONE_EMERGENCY_DEPT.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia15/CoMMpass_IA15_FlatFiles/MMRF_CoMMpass_IA15_PER_PATIENT_VISIT.csv


/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,9,11,12,13,14,15,16,17,18,23,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,47,49,50,51,52,55,57,58,60,61,62,63,64,66,68,70,71,73,75,76,77,78,79,80,83,84,85,87,90,91,92,94,97,98,99,100,101,103,106,107,108,110,113,114,115,117,120,121,122,126,127,128,130,132,133,138,139,141,142,143,144,145,147,148,149,150,151,152,153,155,156,157,158,159,160,161,162,163,165,166,167,168,169,170,177,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,218,224,225,226,227,232,233,242,245,248,251,274,279,291,292,378,380,421,424,426,428,431,433,435,438,440,442,445,447,449,450,451,453,455,457,458,459,460,461,462,463,464,465,466,467,468,469,470,472,475,478,480,482,485,487,489,492,494,496,499,501,503,506,508,510,513,515,517,520,522,524,527,529,531,534,536,538,541,543,545

/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia15/CoMMpass_IA15_FlatFiles/MMRF_CoMMpass_IA15_STAND_ALONE_TRTRESP.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia15/CoMMpass_IA15_FlatFiles/MMRF_CoMMpass_IA15_STAND_ALONE_TREATMENT_REGIMEN.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia15/CoMMpass_IA15_FlatFiles/MMRF_CoMMpass_IA15_STAND_ALONE_AE.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia15/CoMMpass_IA15_FlatFiles/MMRF_CoMMpass_IA15_STAND_ALONE_FAMHX.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia15/CoMMpass_IA15_FlatFiles/MMRF_CoMMpass_IA15_STAND_ALONE_MEDHX.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia15/CoMMpass_IA15_FlatFiles/MMRF_CoMMpass_IA15_STAND_ALONE_ADMISSIONS.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia15/CoMMpass_IA15_FlatFiles/MMRF_CoMMpass_IA15_STAND_ALONE_SURVIVAL.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_mye

/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (24,28,44,74,75,80,81,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
FDIR_sh = '/'.join(FDIR.split('/')[:-1])
data_files[f'Seq_QC_Summary'] = pd.read_csv(os.path.join(FDIR_sh,f'MMRF_CoMMpass_{ia_version.upper()}_Seq_QC_Summary.csv'), delimiter=',', encoding='latin-1')
data_files[f'E74GTF_Cufflinks_Gene_FPKM'] = pd.read_csv(os.path.join(FDIR_sh,f'MMRF_CoMMpass_{ia_version.upper()}a_E74GTF_Cufflinks_Gene_FPKM.txt'), sep='\t', header=None)

/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,1

In [5]:
fpkm_df = data_files['E74GTF_Cufflinks_Gene_FPKM']
if ia_version == 'ia15': 
    fpkm_df.drop(1, inplace=True, axis=1) # drop gene location column
fpkm_df

,0,2,3,4,5,6,7,8,9,10,...,913,914,915,916,917,918,919,920,921,922
0,GENE_ID,MMRF_2689_1_BM,MMRF_2412_2_BM,MMRF_2829_1_BM,MMRF_1817_1_BM,MMRF_1881_1_BM,MMRF_1519_1_BM,MMRF_1988_1_BM,MMRF_2436_1_BM,MMRF_2621_1_BM,...,MMRF_1030_4_BM,MMRF_1659_1_BM,MMRF_1107_1_BM,MMRF_1664_1_BM,MMRF_1338_1_BM,MMRF_2353_1_BM,MMRF_1201_1_BM,MMRF_1510_1_BM,MMRF_1169_1_BM,MMRF_2639_1_BM
1,ENSG00000000003,3.6737900000000003,0.383243,5.401009999999999,25.3442,0.06659989999999999,1.33093,3.9704599999999997,0.122314,0.0300618,...,1.05411,0.574166,21.7797,3.9319300000000004,7.484960000000001,3.7069900000000002,5.43158,5.973409999999999,5.14902,4.59211
2,ENSG00000000005,0.0,0.0,0.0,0.0,0.0,0.0,0.016299200000000003,0.0,0.0,...,0.102339,0.0,0.0,0.07687970000000001,0.0,0.0,0.0,0.0,0.134285,0.0
3,ENSG00000000419,21.5371,30.3897,52.2825,31.316999999999997,32.3215,42.1259,42.2072,37.5501,37.94,...,31.9111,21.0612,31.3742,36.1529,39.5279,33.9686,61.6356,39.9638,35.8638,52.3164
4,ENSG00000000457,3.1828,8.52545,5.863630000000001,8.94431,5.1052,6.38826,4.925619999999999,6.08096,8.09729,...,7.008710000000001,6.06882,7.9908,5.63011,5.38075,5.20582,3.4286300000000005,5.68724,7.173019999999999,1.7283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58091,ERCC-00164,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58092,ERCC-00165,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58093,ERCC-00168,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58094,ERCC-00170,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
design_df = data_files['Seq_QC_Summary']
design_df['sampleID'] = design_df['QC Link SampleName'].apply(lambda x: x.split('_')).apply(lambda x: '_'.join(x[:4]))
design_df['cellType'] = design_df['QC Link SampleName'].apply(lambda x: x.split('_')).apply(lambda x: '_'.join(x[3:5]))
design_df

,Patients::KBase_Patient_ID,Visits::Study Visit ID,Visits::Reason_For_Collection,QC Link SampleName,Batch,Creation Date,Process Date,MMRF_Release_Status,Aliquot_Quantity_to_Send,AliquotStock_BioA_Conc,...,QC_Percent_IgK,QC_Percent_IgL,QC_Percent_Immunoglobulin,QC_Percent_Mapped,QC_Percent_Mitochondrial,QC_Percent_Q20_Bases,QC_Percent_Uniq_Mapping,QC_Read_Counts,sampleID,cellType
0,MMRF_1252,MMRF_1252_1,Baseline,MMRF_1252_1_BM_CD138pos_T2_TSMRU_L00030,9,03/25/2013,03/25/2013,RNA-Yes,2000.0,122.0,...,0.000269,0.346800,0.643451,0.858611154,0.015260,0.748464,0.739704,94429514.0,MMRF_1252_1_BM,BM_CD138pos
1,MMRF_1231,MMRF_1231_1,Baseline,MMRF_1231_1_BM_CD138pos_T2_TSMRU_L00031,9,03/25/2013,03/25/2013,RNA-Yes,2000.0,122.0,...,0.239248,0.004554,0.568413,0.938733108,0.011435,0.839285,0.898018,122973628.0,MMRF_1231_1_BM,BM_CD138pos
2,MMRF_1032,MMRF_1032_1,Baseline,MMRF_1032_1_BM_CD138pos_T2_TSMRU_L00032,9,03/25/2013,03/25/2013,Exclude-ReplicateAssay,500.0,102.0,...,0.004085,0.231458,0.554034,?,0.018662,0.728480,0.737726,NaN,MMRF_1032_1_BM,BM_CD138pos
3,MMRF_1032,MMRF_1032_1,Baseline,MMRF_1032_1_BM_CD138pos_T2_TSMRU_L00033,9,03/25/2013,03/25/2013,RNA-Yes,2000.0,105.0,...,0.004055,0.222092,0.540467,0.924155627,0.019253,0.735184,0.743045,117242528.0,MMRF_1032_1_BM,BM_CD138pos
4,MMRF_1327,MMRF_1327_1,Baseline,MMRF_1327_1_BM_CD138pos_T2_TSMRU_L00034,9,03/25/2013,03/25/2013,RNA-Yes,2000.0,107.0,...,0.155805,0.001158,0.337742,0.925306642,0.033957,0.828943,0.879609,111248848.0,MMRF_1327_1_BM,BM_CD138pos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5397,MMRF_1030,MMRF_1030_5,Confirm Progression,MMRF_1030_5_BM_CD138pos_T6_TSMRU_L22968,50,11/02/2018,11/05/2018,RNA-Yes,500.0,252.0,...,0.144206,0.004825,0.264984,0.976620965,0.028774,0.908586,0.917343,296299912.0,MMRF_1030_5_BM,BM_CD138pos
5398,MMRF_2924,MMRF_2924_1,Baseline,MMRF_2924_1_BM_CD138pos_T1_TSMRU_L22969,50,11/02/2018,11/05/2018,RNA-Yes,500.0,401.0,...,0.345563,0.001026,0.528965,0.925743742,0.018559,0.887871,0.878920,320092158.0,MMRF_2924_1_BM,BM_CD138pos
5399,MMRF_2489,MMRF_2489_2,Confirm Progression,MMRF_2489_2_BM_CD138pos_T6_TSMRU_L22970,50,11/02/2018,11/05/2018,RNA-Yes,500.0,525.0,...,0.169556,0.013399,0.481221,0.937836541,0.022652,0.877533,0.869495,317393404.0,MMRF_2489_2_BM,BM_CD138pos
5400,MMRF_1908,MMRF_1908_4,Confirm Progression,MMRF_1908_4_BM_CD138pos_T2_TSMRU_L22971,50,11/02/2018,11/05/2018,RNA-Yes,500.0,NaN,...,0.281862,0.001507,0.283464,0.972067586,0.025069,0.927645,0.943429,340329486.0,MMRF_1908_4_BM,BM_CD138pos


In [7]:
# make sure that all IDs in FPKM df are in Design df
for val in fpkm_df.head(1).iloc[:,1:].values.squeeze(): 
    if val not in design_df.sampleID.values: 
        assert('fpkm ID not in QC list of IDs')

In [8]:
design_df = design_df[(design_df['cellType'] == 'BM_CD138pos') & (design_df['Visits::Reason_For_Collection'] == 'Baseline') & (design_df['MMRF_Release_Status'] == 'RNA-Yes')]
design_df = design_df[['Patients::KBase_Patient_ID', 'sampleID', 'cellType', 'Visits::Reason_For_Collection', 'Batch', 'QC_Percent_Mitochondrial']]
design_df.rename(columns = {'Patients::KBase_Patient_ID':'public_id', 'Visit::Reason_For_Collection':'visit'}, inplace = True) 
design_df

,public_id,sampleID,cellType,Visits::Reason_For_Collection,Batch,QC_Percent_Mitochondrial
0,MMRF_1252,MMRF_1252_1_BM,BM_CD138pos,Baseline,9,0.015260
1,MMRF_1231,MMRF_1231_1_BM,BM_CD138pos,Baseline,9,0.011435
3,MMRF_1032,MMRF_1032_1_BM,BM_CD138pos,Baseline,9,0.019253
4,MMRF_1327,MMRF_1327_1_BM,BM_CD138pos,Baseline,9,0.033957
5,MMRF_1210,MMRF_1210_1_BM,BM_CD138pos,Baseline,9,0.009158
...,...,...,...,...,...,...
5238,MMRF_2914,MMRF_2914_1_BM,BM_CD138pos,Baseline,49,0.012227
5243,MMRF_2905,MMRF_2905_1_BM,BM_CD138pos,Baseline,49,0.018511
5394,MMRF_2926,MMRF_2926_1_BM,BM_CD138pos,Baseline,50,0.018502
5396,MMRF_2921,MMRF_2921_1_BM,BM_CD138pos,Baseline,50,0.010710


In [9]:
# outcomes 
oc_df = data_files['STAND_ALONE_SURVIVAL'] 
# (90 < oc_df.deathdy) & (oc_df.deathdy <= 730) & (not oc_df.deathdy.isna())
oc_df['mortality_2yr'] = oc_df['deathdy'].apply(lambda x: int(90 < x and x <= 730 and not np.isnan(x)))
oc_df['mortality_1yr'] = oc_df['deathdy'].apply(lambda x: int(90 < x and x <= 364 and not np.isnan(x)))
oc_df = oc_df[['public_id', 'mortality_2yr', 'mortality_1yr']]
oc_df.rename(columns={'public_id': 'PUBLIC_ID'}, inplace=True)
oc_df

/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,PUBLIC_ID,mortality_2yr,mortality_1yr
0,MMRF_1014,0,0
1,MMRF_1017,1,0
2,MMRF_1024,0,0
3,MMRF_1038,0,0
4,MMRF_1033,0,0
...,...,...,...
1138,MMRF_2846,0,0
1139,MMRF_2847,0,0
1140,MMRF_2848,0,0
1141,MMRF_2851,0,0


In [10]:
healthy_mins_max = {
    'cbc_abs_neut':(2., 7.5,1/3.), # abs neutrophil count (3.67, 1.), (2.83, 4.51)
    'chem_albumin':(34, 50,1/8.), # chemical albumin (43.62, 2.77), (41.30, 45.94)
    'chem_bun':(2.5, 7.1,1/5.), #BUN # reference range, (4.8, 1.15)
    'chem_calcium':(2.2, 2.7,2.), #Calcium, (2.45, 0.125)
    'chem_creatinine':(66, 112,1/36.), # creatinine, (83., 24.85), (62.22, 103.77)
    'chem_glucose':(3.9, 6.9,1/5.), # glucose, (4.91, 0.40), (4.58, 5.24)
    'cbc_hemoglobin':(13., 17.,1), # hemoglobin (12.90, 15.64), (8.86, 1.02)
    'chem_ldh':(2.33, 4.67,1/3.), #LDH, (3.5, 0.585)
    'serum_m_protein':(0.1, 1.1, 1), # M protein (<3 g/dL is MGUS, any presence of protein is pathological); am just using the data mean/std for this, (0.85, 1.89)
    'urine_24hr_m_protein':(0.0, 0.1, 1), # Urine M protein 
    'cbc_platelet':(150, 400,1/60.), # platelet count (206.42, 334.57), (270.5, 76.63)
    'chem_totprot':(6, 8,1/6.), # total protein, (7, 0.5)
    'urine_24hr_total_protein':(0, 0.23, 1), # 
    'cbc_wbc':(3, 10,1/4.), # WBC  (5.71, 8.44), (7.07, 1.63)
    'serum_iga':(0.85, 4.99, 1.), # IgA, (2.92, 1.035)
    'serum_igg':(6.10, 16.16,1/10.), # IgG, (11.13, 2.515)
    'serum_igm':(0.35, 2.42,1), #IgM, (1.385, 0.518)
    'serum_lambda':(0.57, 2.63, 1/2.), #serum lambda, (1.6, 0.515)
    'serum_kappa':(.33, 1.94,1/8.), #serum kappa , (1.135, 0.403)
    'serum_beta2_microglobulin':(0.7, 1.80, 1/3.), #serum_beta2_microglobulin,
    'serum_c_reactive_protein':(0.0, 1., 1.) #serum_c_reactive_protein,
}

In [11]:
baseline_labs = ['D_LAB_cbc_abs_neut', 'D_LAB_chem_albumin', 'D_LAB_chem_bun', 'D_LAB_chem_calcium', 'D_LAB_chem_creatinine',
        'D_LAB_chem_glucose', 'D_LAB_cbc_hemoglobin', 'D_LAB_serum_kappa', 'D_LAB_serum_m_protein', 'D_LAB_cbc_platelet',
        'D_LAB_chem_totprot', 'D_LAB_cbc_wbc', 'D_LAB_serum_iga', 'D_LAB_serum_igg', 'D_LAB_serum_igm', 'D_LAB_serum_beta2_microglobulin',
        'D_LAB_serum_lambda']
pt_feats = ['D_PT_race', 'D_PT_ethnic', 'D_PT_gender', 'D_PT_age', 'D_PT_iss', 'line1sct','ecog']
ss_feats = ['SS_DOESTHEPATIEN', 'SS_SPINALCORDCOM', 'SS_BONEPAIN', 'SS_FATIGUE', 'SS_HYPERCALCEMIA', 'SS_RENALINSUFFIC', 'SS_ANEMIAHEMOGLO', 'SS_BONELESIONSLY', 
            'SS_BONELESIONSOS', 'SS_SOFTTISSUEPLA', 'SS_OFTTISSUEPLAS', 'SS_RECURRENTBACT', 'SS_SYMPTOMATICHY', 'SS_AMYLOIDOSIS']

# D_PT
baseline_df = data_files['PER_PATIENT']
baseline_df = baseline_df[['PUBLIC_ID'] + pt_feats]
baseline_df.fillna(baseline_df.mean(0), axis=0, inplace=True)
baseline_df.loc[:,'D_PT_age'] = (baseline_df.loc[:,'D_PT_age'].values - np.mean(baseline_df.loc[:,'D_PT_age'].values)) / np.std(baseline_df.loc[:,'D_PT_age'].values)
baseline_df.loc[:,'D_PT_gender'] = (baseline_df.loc[:,'D_PT_gender'].values*2)-3

# D_LAB
ppv_df = data_files['PER_PATIENT_VISIT']
labs_df= ppv_df.filter(regex='^D_LAB|PUBLIC_ID|VISIT')
bl_df  = labs_df[labs_df.VISIT==0].reset_index(drop=True)
bl_df  = bl_df.groupby('PUBLIC_ID').mean()
bl_df.fillna(bl_df.mean(0), axis=0, inplace=True)
bl_df  = bl_df.filter(items=['PUBLIC_ID']+baseline_labs)
bl_df.rename(columns = dict([(k,k.split('D_LAB_')[1]) for k in baseline_labs]), inplace=True)
baseline_labs = np.array([k.split('D_LAB_')[1] for k in baseline_labs])
medians = bl_df[baseline_labs].median(0)
maxval  = (5*(1+medians))
clipped = bl_df[baseline_labs].clip(upper = maxval, axis=1)
print ('parse_labs: clipped values to 5x median (before/after)\n',pd.concat([bl_df[baseline_labs].max(0), clipped[baseline_labs].max(0)],axis=1))
bl_df.loc[:,baseline_labs] = clipped

for lab in baseline_labs: 
    print(f'normalizing lab: [{lab}]')
    lmin, lmax, scale = healthy_mins_max[lab]
    bl_df[lab] = (bl_df[lab].values - lmax)*scale

# (new_dset['labs_data_clean'][...,idx]-healthy_max)*scale
# D_TRI_CF
pp_df  = data_files['PER_PATIENT']
tri_df = ppv_df.filter(regex='^D_TRI_CF_ABNORMALITY|PUBLIC_ID|VISIT')
tri_df = tri_df[tri_df.VISIT==0].reset_index(drop=True)
for col in tri_df.columns.values: 
    if 'ABNORMALITY' in col:         
        tri_df.loc[tri_df[col] == 'No', [col]] = int(0)
        tri_df.loc[(tri_df[col] == 'Yes'), [col]] = int(1)
        tri_df.loc[(tri_df[col] == 'Not Done') | (tri_df[col] == 'Checked'), [col]] = np.nan
# tri_df  = tri_df.groupby('PUBLIC_ID').mean()
tri_df  = tri_df.filter(regex='^D_TRI_CF_ABNORMALITY|PUBLIC_ID').infer_objects()
tri_df  = tri_df.groupby('PUBLIC_ID').mean()
tri_df.fillna(tri_df.mean(0), axis=0, inplace=True)
tri_df

# D_SS 
ppv_df = data_files['PER_PATIENT_VISIT']
ss_df  = ppv_df[['PUBLIC_ID', 'VISIT'] + ss_feats]
ss_df  = ss_df[ss_df.VISIT==0].reset_index(drop=True)
for col in ss_df.columns.values: 
    if 'SS' in col and col != 'SS_DOESTHEPATIEN': 
        ss_df.loc[ss_df[col] == 'Checked', [col]] = 1
        for i, row in ss_df.iterrows(): 
            if (row['SS_DOESTHEPATIEN'] == 'No' or row['SS_DOESTHEPATIEN'] == 'Yes') \
                and math.isnan(ss_df.loc[i,col]): 
                    ss_df.loc[i,col] = 0
ss_df  = ss_df.filter(regex='^SS_|PUBLIC_ID').infer_objects()
ss_df  = ss_df.groupby('PUBLIC_ID').mean()
ss_df.fillna(ss_df.mean(0), axis=0, inplace=True)
ss_df

/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


parse_labs: clipped values to 5x median (before/after)
                                     0        1
cbc_abs_neut                   16.512   16.512
chem_albumin                   54.000   54.000
chem_bun                       52.122   44.270
chem_calcium                    4.250    4.250
chem_creatinine              1986.348  451.420
chem_glucose                   16.940   16.940
cbc_hemoglobin                  9.982    9.982
serum_kappa                455000.000   67.500
serum_m_protein                12.270   12.270
cbc_platelet                  668.000  668.000
chem_totprot                   17.100   17.100
cbc_wbc                        34.600   34.600
serum_iga                     276.630    8.400
serum_igg                     147.000  147.000
serum_igm                      84.200    6.250
serum_beta2_microglobulin      37.900   23.065
serum_lambda                46200.000   11.900
normalizing lab: [cbc_abs_neut]
normalizing lab: [chem_albumin]
normalizing lab: [chem_bun]
normal

,SS_SPINALCORDCOM,SS_BONEPAIN,SS_FATIGUE,SS_HYPERCALCEMIA,SS_RENALINSUFFIC,SS_ANEMIAHEMOGLO,SS_BONELESIONSLY,SS_BONELESIONSOS,SS_SOFTTISSUEPLA,SS_OFTTISSUEPLAS,SS_RECURRENTBACT,SS_SYMPTOMATICHY,SS_AMYLOIDOSIS
PUBLIC_ID,,,,,,,,,,,,,
MMRF_1007,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
MMRF_1011,0.000000,1.00000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
MMRF_1013,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000
MMRF_1014,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
MMRF_1016,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
MMRF_2846,0.071061,0.53862,0.392379,0.065911,0.178167,0.451081,0.601442,0.147271,0.107106,0.023687,0.027806,0.00309,0.014418
MMRF_2847,0.071061,0.53862,0.392379,0.065911,0.178167,0.451081,0.601442,0.147271,0.107106,0.023687,0.027806,0.00309,0.014418
MMRF_2848,0.071061,0.53862,0.392379,0.065911,0.178167,0.451081,0.601442,0.147271,0.107106,0.023687,0.027806,0.00309,0.014418


In [12]:
# pd.merged(baseline_df, bl_df, tri_df, ss_df)
merged= pd.merge(baseline_df, bl_df, on='PUBLIC_ID', sort=True)
merged= pd.merge(merged, tri_df, on='PUBLIC_ID', sort=True)
merged= pd.merge(merged, ss_df, on='PUBLIC_ID', sort=True)
merged

,PUBLIC_ID,D_PT_race,D_PT_ethnic,D_PT_gender,D_PT_age,D_PT_iss,line1sct,ecog,cbc_abs_neut,chem_albumin,...,SS_HYPERCALCEMIA,SS_RENALINSUFFIC,SS_ANEMIAHEMOGLO,SS_BONELESIONSLY,SS_BONELESIONSOS,SS_SOFTTISSUEPLA,SS_OFTTISSUEPLAS,SS_RECURRENTBACT,SS_SYMPTOMATICHY,SS_AMYLOIDOSIS
0,MMRF_1007,1.000000,2.000000,-1,1.427714,3.0,0,1.000000,-0.800000,-2.1250,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,MMRF_1011,1.000000,2.000000,-1,-1.221744,3.0,1,1.000000,0.260000,-3.1250,...,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
2,MMRF_1013,1.000000,2.000000,1,0.292232,3.0,1,1.000000,-1.056667,-1.2500,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000
3,MMRF_1014,1.000000,2.000000,1,0.102985,1.0,1,0.000000,-2.066667,-1.1250,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,MMRF_1016,1.000000,2.000000,-1,-0.654003,1.0,0,1.000000,-0.566667,-1.7500,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,MMRF_2846,1.475798,1.956746,1,-0.180886,3.0,1,0.879147,-1.666667,-2.2500,...,0.065911,0.178167,0.451081,0.601442,0.147271,0.107106,0.023687,0.027806,0.00309,0.014418
1139,MMRF_2847,1.475798,1.956746,-1,-0.559380,3.0,0,0.879147,-0.370000,-1.8875,...,0.065911,0.178167,0.451081,0.601442,0.147271,0.107106,0.023687,0.027806,0.00309,0.014418
1140,MMRF_2848,1.475798,1.956746,1,-1.694862,2.0,1,0.879147,2.900000,-2.4375,...,0.065911,0.178167,0.451081,0.601442,0.147271,0.107106,0.023687,0.027806,0.00309,0.014418
1141,MMRF_2851,1.475798,1.956746,1,-0.086262,1.0,1,0.879147,-0.900000,-1.5125,...,0.065911,0.178167,0.451081,0.601442,0.147271,0.107106,0.023687,0.027806,0.00309,0.014418


In [13]:
df_pp = data_files['PER_PATIENT']; granularity = 60
df_restricted = df_pp[['PUBLIC_ID','D_PT_lstalive', 'D_PT_lastdy', 'D_PT_deathdy']]

# Uncensored outcomes
# First select patients who have the event (death)
df_death = df_restricted.dropna(subset=['D_PT_deathdy'])
assert (df_death.PUBLIC_ID.unique().shape[0]==df_death.shape[0]),'duplicates found'
death_vals = df_death[['PUBLIC_ID','D_PT_deathdy']].values
pids_y1  = death_vals[:,0]
y1       = death_vals[:,1]/float(granularity)
e1       = np.ones_like(y1)
df_death['eventdy'] = df_death.loc[:,'D_PT_deathdy']/float(granularity)
df_ttm1 = df_death[['PUBLIC_ID', 'eventdy']]

df_subset            = df_restricted[~df_restricted['PUBLIC_ID'].isin(pids_y1.ravel().tolist())].reset_index(drop=True)
df_alive_or_censored = df_subset[['PUBLIC_ID','D_PT_lstalive','D_PT_lastdy']]
df_alive_or_censored = df_alive_or_censored.fillna(0)
df_alive_or_censored['censored_date'] = df_alive_or_censored[['D_PT_lstalive', 'D_PT_lastdy']].values.max(1)
df_alive_or_censored = df_alive_or_censored[['PUBLIC_ID','censored_date']]
df_alive_or_censored = df_alive_or_censored[df_alive_or_censored['censored_date']>0].reset_index(drop=True)
df_alive_or_censored['eventdy'] = df_alive_or_censored.loc[:,'censored_date']/float(granularity)
df_ttm2 = df_alive_or_censored[['PUBLIC_ID', 'eventdy']]

df_ttm = pd.concat([df_ttm1, df_ttm2], axis=0, sort=True)
df_ttm['eventdy'] = (df_ttm.loc[:,'eventdy'].values - np.mean(df_ttm.loc[:,'eventdy'].values)) / np.std(df_ttm.loc[:,'eventdy'].values)
df_ttm

/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,PUBLIC_ID,eventdy
1,MMRF_1017,-1.100156
3,MMRF_1038,0.893260
5,MMRF_1007,-1.389962
6,MMRF_1052,-1.410776
7,MMRF_1082,-0.584590
...,...,...
701,MMRF_2808,-1.620526
702,MMRF_2833,-1.662155
703,MMRF_2838,-1.534064
704,MMRF_2839,-1.843084


In [14]:
# df_ttm, merged, oc_df (design_df)
design_df = design_df.rename(columns={'public_id': 'PUBLIC_ID'})
design_df = pd.merge(design_df, merged, how='inner', on='PUBLIC_ID')
design_df = pd.merge(design_df, oc_df, how='inner', on='PUBLIC_ID')
design_df = pd.merge(design_df, df_ttm, how='inner', on='PUBLIC_ID')

In [15]:
design_df

,PUBLIC_ID,sampleID,cellType,Visits::Reason_For_Collection,Batch,QC_Percent_Mitochondrial,D_PT_race,D_PT_ethnic,D_PT_gender,D_PT_age,...,SS_BONELESIONSLY,SS_BONELESIONSOS,SS_SOFTTISSUEPLA,SS_OFTTISSUEPLAS,SS_RECURRENTBACT,SS_SYMPTOMATICHY,SS_AMYLOIDOSIS,mortality_2yr,mortality_1yr,eventdy
0,MMRF_1252,MMRF_1252_1_BM,BM_CD138pos,Baseline,9,0.015260,1.000000,2.000000,1,1.333090,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1,0,-1.172207
1,MMRF_1231,MMRF_1231_1_BM,BM_CD138pos,Baseline,9,0.011435,2.000000,2.000000,1,1.333090,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,0,-1.777437
2,MMRF_1032,MMRF_1032_1_BM,BM_CD138pos,Baseline,9,0.019253,1.000000,2.000000,-1,1.238467,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0,0,-0.418071
3,MMRF_1327,MMRF_1327_1_BM,BM_CD138pos,Baseline,9,0.033957,1.000000,2.000000,1,0.765349,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,0,-1.666958
4,MMRF_1210,MMRF_1210_1_BM,BM_CD138pos,Baseline,9,0.009158,6.000000,3.000000,-1,0.386855,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,0,1.629784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,MMRF_2746,MMRF_2746_1_BM,BM_CD138pos,Baseline,44,0.034043,1.475798,1.956746,1,-2.167979,...,0.601442,0.147271,0.107106,0.023687,0.027806,0.00309,0.014418,0,0,-1.396366
663,MMRF_2761,MMRF_2761_1_BM,BM_CD138pos,Baseline,45,0.016372,1.475798,1.956746,-1,-0.086262,...,0.601442,0.147271,0.107106,0.023687,0.027806,0.00309,0.014418,0,0,-1.899123
664,MMRF_2781,MMRF_2781_1_BM,BM_CD138pos,Baseline,45,0.049270,1.475798,1.956746,1,0.197608,...,0.601442,0.147271,0.107106,0.023687,0.027806,0.00309,0.014418,1,1,-1.580497
665,MMRF_2808,MMRF_2808_1_BM,BM_CD138pos,Baseline,45,0.011867,1.475798,1.956746,-1,-0.086262,...,0.601442,0.147271,0.107106,0.023687,0.027806,0.00309,0.014418,0,0,-1.620526


In [26]:
design_df.sampleID.values
# fpkm_df.loc[fpkm_df.loc[0,:] in design_df.sampleID.values, list(fpkm_df.loc[0,:].values)]
fpkm_df

,0,2,3,4,5,6,7,8,9,10,...,913,914,915,916,917,918,919,920,921,922
0,GENE_ID,MMRF_2689_1_BM,MMRF_2412_2_BM,MMRF_2829_1_BM,MMRF_1817_1_BM,MMRF_1881_1_BM,MMRF_1519_1_BM,MMRF_1988_1_BM,MMRF_2436_1_BM,MMRF_2621_1_BM,...,MMRF_1030_4_BM,MMRF_1659_1_BM,MMRF_1107_1_BM,MMRF_1664_1_BM,MMRF_1338_1_BM,MMRF_2353_1_BM,MMRF_1201_1_BM,MMRF_1510_1_BM,MMRF_1169_1_BM,MMRF_2639_1_BM
1,ENSG00000000003,3.6737900000000003,0.383243,5.401009999999999,25.3442,0.06659989999999999,1.33093,3.9704599999999997,0.122314,0.0300618,...,1.05411,0.574166,21.7797,3.9319300000000004,7.484960000000001,3.7069900000000002,5.43158,5.973409999999999,5.14902,4.59211
2,ENSG00000000005,0.0,0.0,0.0,0.0,0.0,0.0,0.016299200000000003,0.0,0.0,...,0.102339,0.0,0.0,0.07687970000000001,0.0,0.0,0.0,0.0,0.134285,0.0
3,ENSG00000000419,21.5371,30.3897,52.2825,31.316999999999997,32.3215,42.1259,42.2072,37.5501,37.94,...,31.9111,21.0612,31.3742,36.1529,39.5279,33.9686,61.6356,39.9638,35.8638,52.3164
4,ENSG00000000457,3.1828,8.52545,5.863630000000001,8.94431,5.1052,6.38826,4.925619999999999,6.08096,8.09729,...,7.008710000000001,6.06882,7.9908,5.63011,5.38075,5.20582,3.4286300000000005,5.68724,7.173019999999999,1.7283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58091,ERCC-00164,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58092,ERCC-00165,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58093,ERCC-00168,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58094,ERCC-00170,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
